In [2]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import Counter


In [8]:
# unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
last_token_before_wo_wait = torch.load('tmp.pt')


/tmp/ipykernel_211507/1185413857.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  last_token_before_wo_wait = torch.load('tmp.pt')


In [10]:
rank_list = []
layers = 29
numerical_ranks = []
numerical_ranks_no_wait = []
for layer in tqdm(range(layers)):
    token_before_wait = torch.cat([
        x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
        for x in last_token_before_wo_wait
    ]).squeeze().to("cuda:0")
    words = []
    for token in token_before_wait:
        logits = torch.matmul(token, unembedding_matrix.T)
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        rank = torch.argsort(log_probs, dim=-1, descending=True)[0].item()
        words.append(rank)
    
    # token_before_no_wait = torch.cat([
    #     x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
    #     for x in last_token_before_no_wait
    # ]).squeeze().to("cuda:0").to(torch.float32)
    # Apply logit lens for the current layer
    # logits = torch.matmul(token_before_wait, unembedding_matrix.T)
    # log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    # rank = torch.argsort(log_probs, dim=-1, descending=True)
    counter = Counter(words)

    top3 = counter.most_common(3)
    rank_list.append(top3)


100%|██████████| 29/29 [05:56<00:00, 12.28s/it]


In [23]:
print(len(last_token_before_wo_wait))

419


In [11]:
print(rank_list)

[[(3559, 9647), (4510, 4), (526, 3)], [(4028, 9655), (87965, 12), (130457, 11)], [(4028, 9683), (87965, 3), (68106, 2)], [(4028, 9683), (133129, 3), (121555, 1)], [(4028, 9675), (133129, 7), (121555, 3)], [(4028, 9682), (133129, 7)], [(4028, 9688), (133129, 1)], [(4028, 9686), (133129, 3)], [(4028, 9662), (71720, 12), (53566, 4)], [(4028, 9672), (71720, 6), (133129, 3)], [(4028, 9684), (133129, 3), (87965, 1)], [(4028, 9665), (133129, 13), (87965, 6)], [(4028, 9644), (133129, 36), (71720, 5)], [(4028, 9616), (133129, 42), (71720, 11)], [(4028, 9567), (71720, 51), (133129, 33)], [(4028, 9657), (71720, 16), (133129, 12)], [(4028, 9619), (71720, 44), (133129, 15)], [(4028, 9637), (71720, 30), (133129, 13)], [(4028, 9646), (71720, 25), (133129, 13)], [(4028, 9502), (71720, 86), (133129, 68)], [(4028, 9596), (133129, 67), (71720, 24)], [(4028, 9541), (133129, 129), (71720, 18)], [(4028, 9451), (133129, 223), (29193, 11)], [(4028, 9410), (133129, 266), (71720, 7)], [(4028, 9542), (133129, 14

In [12]:
# Load the tokenizer (already imported from transformers)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
char_list = []
for layer, tokens in enumerate(rank_list):
    char_list.append([])
    for token in tokens:
        char_list[layer].append(tokenizer.decode(token[0]))
print(char_list)

[['swer', ' py', ' int'], ['\ufeffusing', '"crypto', 'tör'], ['\ufeffusing', '"crypto', " './../../"], ['\ufeffusing', 'โปรแ', '鋆'], ['\ufeffusing', 'โปรแ', '鋆'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', '"sync', '>NN'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', 'โปรแ', '"crypto'], ['\ufeffusing', 'โปรแ', '"crypto'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', ' similarly'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' )\n\n\n\n\n\n\n\n'], ['So', '    ', 'But']]


In [13]:
for idx, char_ in enumerate(char_list):
    print(f"layer{idx}: {char_}")

layer0: ['swer', ' py', ' int']
layer1: ['\ufeffusing', '"crypto', 'tör']
layer2: ['\ufeffusing', '"crypto', " './../../"]
layer3: ['\ufeffusing', 'โปรแ', '鋆']
layer4: ['\ufeffusing', 'โปรแ', '鋆']
layer5: ['\ufeffusing', 'โปรแ']
layer6: ['\ufeffusing', 'โปรแ']
layer7: ['\ufeffusing', 'โปรแ']
layer8: ['\ufeffusing', '"sync', '>NN']
layer9: ['\ufeffusing', '"sync', 'โปรแ']
layer10: ['\ufeffusing', 'โปรแ', '"crypto']
layer11: ['\ufeffusing', 'โปรแ', '"crypto']
layer12: ['\ufeffusing', 'โปรแ', '"sync']
layer13: ['\ufeffusing', 'โปรแ', '"sync']
layer14: ['\ufeffusing', '"sync', 'โปรแ']
layer15: ['\ufeffusing', '"sync', 'โปรแ']
layer16: ['\ufeffusing', '"sync', 'โปรแ']
layer17: ['\ufeffusing', '"sync', 'โปรแ']
layer18: ['\ufeffusing', '"sync', 'โปรแ']
layer19: ['\ufeffusing', '"sync', 'โปรแ']
layer20: ['\ufeffusing', 'โปรแ', '"sync']
layer21: ['\ufeffusing', 'โปรแ', '"sync']
layer22: ['\ufeffusing', 'โปรแ', ' similarly']
layer23: ['\ufeffusing', 'โปรแ', '"sync']
layer24: ['\ufeffusing', 'โปร

In [7]:
pos_char_list = [['swer', 'AVA'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' similarly'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['Wait', 'But', 'So']]

for idx, char_ in enumerate(pos_char_list):
    print(f"layer{idx}: {char_}")

layer0: ['swer', 'AVA']
layer1: ['\ufeffusing']
layer2: ['\ufeffusing']
layer3: ['\ufeffusing']
layer4: ['\ufeffusing']
layer5: ['\ufeffusing', 'โปรแ']
layer6: ['\ufeffusing']
layer7: ['\ufeffusing']
layer8: ['\ufeffusing']
layer9: ['\ufeffusing']
layer10: ['\ufeffusing']
layer11: ['\ufeffusing', 'โปรแ']
layer12: ['\ufeffusing', 'โปรแ']
layer13: ['\ufeffusing', 'โปรแ', ' Diễn']
layer14: ['\ufeffusing', 'โปรแ', ' Diễn']
layer15: ['\ufeffusing']
layer16: ['\ufeffusing', 'โปรแ']
layer17: ['\ufeffusing']
layer18: ['\ufeffusing']
layer19: ['\ufeffusing', 'โปรแ', '"sync']
layer20: ['\ufeffusing', 'โปรแ']
layer21: ['\ufeffusing', 'โปรแ']
layer22: ['\ufeffusing', 'โปรแ', ' similarly']
layer23: ['\ufeffusing', 'โปรแ']
layer24: ['\ufeffusing', 'โปรแ']
layer25: ['\ufeffusing', 'โปรแ']
layer26: ['\ufeffusing', 'โปรแ']
layer27: ['\ufeffusing', 'โปรแ']
layer28: ['Wait', 'But', 'So']


In [3]:
last_token_before_wait = torch.load('last_token_before_wait2.pt')
# last_token_before_no_wait = torch.load('last_token_before_wo_wait.pt')
unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
# last_token_before_wo_wait = torch.load('last_token_before_wo_wait2.pt')

/tmp/ipykernel_211507/988955994.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  last_token_before_wait = torch.load('last_token_before_wait2.pt')
/tmp/ipykernel_211507/9

In [4]:
rank_list = []
layers = 29
numerical_ranks = []
numerical_ranks_no_wait = []
for layer in tqdm(range(layers)):
    token_before_wait = torch.cat([
        x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
        for x in last_token_before_wait
    ]).squeeze().to("cuda:0")
    # token_before_no_wait = torch.cat([
    #     x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
    #     for x in last_token_before_no_wait
    # ]).squeeze().to("cuda:0").to(torch.float32)
    # Apply logit lens for the current layer
    logits = torch.matmul(token_before_wait, unembedding_matrix.T)
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    rank = torch.argsort(log_probs, dim=-1, descending=True)
    counter = Counter(rank[:, 0].cpu().numpy())

    top3 = counter.most_common(3)
    rank_list.append(top3)


100%|██████████| 29/29 [00:01<00:00, 20.89it/s]


In [5]:
print(rank_list)

[[(3559, 1981), (35610, 1)], [(4028, 1982)], [(4028, 1982)], [(4028, 1982)], [(4028, 1982)], [(4028, 1980), (133129, 2)], [(4028, 1982)], [(4028, 1982)], [(4028, 1982)], [(4028, 1982)], [(4028, 1982)], [(4028, 1981), (133129, 1)], [(4028, 1979), (133129, 3)], [(4028, 1975), (133129, 6), (141351, 1)], [(4028, 1979), (133129, 2), (141351, 1)], [(4028, 1982)], [(4028, 1981), (133129, 1)], [(4028, 1982)], [(4028, 1982)], [(4028, 1959), (133129, 18), (71720, 3)], [(4028, 1966), (133129, 16)], [(4028, 1971), (133129, 11)], [(4028, 1966), (133129, 15), (29193, 1)], [(4028, 1973), (133129, 9)], [(4028, 1976), (133129, 6)], [(4028, 1975), (133129, 7)], [(4028, 1954), (133129, 28)], [(4028, 1969), (133129, 13)], [(14190, 1515), (3983, 151), (4416, 79)]]


In [6]:
# Load the tokenizer (already imported from transformers)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
char_list = []
for layer, tokens in enumerate(rank_list):
    char_list.append([])
    for token in tokens:
        char_list[layer].append(tokenizer.decode(token[0]))
print(char_list)

[['swer', 'AVA'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' similarly'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['Wait', 'But', 'So']]


In [ ]:
print(wait_list)

[11489, 14190, 3783, 13824]


In [ ]:
tokenizer.decode([rank_list[0][0][0], rank_list[1][0][0], rank_list[2][0][0]])

'.\n\n.\n\n.\n\n'

In [ ]:
tokenizer.additional_special_tokens

[]

In [26]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# TODO: why wait and space wait is different, why .\n\n is a token instead of .

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [27]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):